In [ ]:
import numpy as np
import pandas as pd
import os 
import matplotlib.pyplot as plt
# import umap
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib

from scroutines import config_plots
from scroutines import basicu
from scroutines import miscu
# from dredFISH.Utils import __init__plots
# from dredFISH.Utils import basicu
# from dredFISH.Utils import miscu

In [ ]:
import sys
sys.setrecursionlimit(40000)
from scipy.cluster import hierarchy as sch

In [ ]:
pth_dat = '/u/home/f/f7xiesnm/project-zipursky/easifish/sparse06/proc/r1'
!ls $pth_dat

In [ ]:
gene_names = {
    'r1_c0': "Sorcs3",
    'r1_c2': "Cdh13",
    'r1_c4': "Chrm2",
    
    
}
channel_names = pd.Series(gene_names).reset_index().set_index(0)['index']
ftrs_order = list(gene_names.keys())

genes_order = [gene_names[ftr] for ftr in ftrs_order]
genes_order

In [ ]:
f_meta = os.path.join(pth_dat, 'roi.csv')
# f_ftrs = os.path.join(pth_dat, 'spotcount_merged.csv')
f_ftrs = os.path.join(pth_dat, 'spotcount.csv')

In [ ]:
f_gcamp = f'/u/home/f/f7xiesnm/project-zipursky/easifish/sparse06/outputs/r1/intensities/r1_c1_intensity.csv' 
df_gcamp = pd.read_csv(f_gcamp).rename(columns={'mean_intensity': 'gcamp'})
df_gcamp

In [ ]:
meta = pd.read_csv(f_meta, index_col=0)
ftrs = pd.read_csv(f_ftrs, index_col=0)
df = ftrs.join(meta).dropna()
ftrs_cols = ftrs.columns.values
print(meta.shape, ftrs.shape, df.shape)

In [ ]:
df = df.join(df_gcamp.set_index('roi')[['gcamp']])
df['gcamp_binary'] = (df['gcamp'] > 150).astype(int)
df

In [ ]:
# check dim
print(1920*5*0.23/2) # in um preexpansion
print(1920*4*0.23/2)

In [ ]:
max_x, max_y, max_z = df[['x', 'y', 'z']].describe().loc['max']
min_x, min_y, min_z = df[['x', 'y', 'z']].describe().loc['min']
print(f"x: {min_x:.1f}, {max_x:.1f}")
print(f"y: {min_y:.1f}, {max_y:.1f}")
print(f"z: {min_z:.1f}, {max_z:.1f}")
df['to_edge'] = np.minimum(
    np.minimum(df['x']-min_x, max_x-df['x']), 
    np.minimum(df['y']-min_y, max_y-df['y']), 
    np.minimum(df['z']-min_z, max_z-df['z']),
)

for col in ftrs_cols:
    val = df[col].values
    print(f'{col}: {np.min(val):.0f}, {np.max(val):.0f}, {np.sum(val>0)/len(df):.2f}')

In [ ]:
########### FLIP z!
df['z'] = max_z -df['z']

# check basics

In [ ]:
x = df['x'].values
y = df['y'].values
z = df['z'].values

fig, axs = plt.subplots(3, 1, figsize=(1*6, 3*3))
ax = axs[0]
sns.histplot(x, ax=ax)
ax.set_xlabel('x')

ax = axs[1]
sns.histplot(y, ax=ax)
ax.set_xlabel('y')

ax = axs[2]
sns.histplot(z, ax=ax)
ax.set_xlabel('z')

fig.subplots_adjust(hspace=0.5)
plt.show()

fig, ax = plt.subplots(1, 1, figsize=(1*6, 1*3))
sns.histplot(df['area'].values, ax=ax)
ax.set_xlabel('area')

In [ ]:
# remove outliers
conds = [
    df['area'] > 500,
    df['x'] > min_x + 100,
    df['x'] < max_x - 100,
    
    df['y'] > min_y + 100,
    df['y'] < max_y - 100,
    
    df['z'] > min_z + 20,
    df['z'] < max_z - 20,
]
cond_all = np.ones(len(df)) > 0
for cond in conds:
    cond_all = np.logical_and(cond_all, cond)
    print(cond_all.sum())
print(len(df), cond_all.sum())

dfnew = df[cond_all].copy()

# size
med_size = dfnew['area'].median()
print(med_size)
size_factor = dfnew['area']/med_size

# norm by size
mat_raw = dfnew[ftrs_cols].values
dfnew[ftrs_cols] = dfnew[ftrs_cols].divide(size_factor, axis=0)
mat_sizenorm = dfnew[ftrs_cols].values

# log2+1 norm
normed_ftrs_cols = np.char.add(ftrs_cols.astype(str) ,'_log2')
dfnew[normed_ftrs_cols] = np.log2(dfnew[ftrs_cols]+1)
mat_log2norm = dfnew[normed_ftrs_cols].values

# zscore
mat_zscore = (mat_log2norm - np.mean(mat_log2norm, axis=0))/np.std(mat_log2norm, axis=0)

In [ ]:
mat_ftrs = dfnew.filter(regex='_log2$') #.filter(regex='^((?!r1).)*$') #.columns
mat_ftrs.columns = [gene_names[col[:-len('_log2')]] for col in mat_ftrs.columns]
mat_ftrs = basicu.zscore(mat_ftrs, axis=0)
mat_ftrs

In [ ]:
# mat_cre = dfnew[['r1_c0_log2', 'r1_c2_log2']]
# mat_cre.columns = [gene_names[col[:-len('_log2')]] for col in mat_cre.columns]
# mat_cre

In [ ]:
# # PCAs
# n_pc = len(genes_order)
# idx_pcs = np.arange(1, n_pc+1)

# pca = PCA(n_components=n_pc)
# pcs = pca.fit_transform(mat_zscore) 
# plt.plot(idx_pcs, pca.explained_variance_ratio_, 'o')
# plt.xticks(idx_pcs)
# plt.ylim(ymin=0)
# plt.axhline(1/n_pc, color='k')
# plt.xlabel('PC')
# plt.ylabel('Explained variance')

# for i in idx_pcs:
#     dfnew[f'PC{i}'] = pcs[:,i-1]

In [ ]:
# # UMAPs
# Zspace = umap.UMAP(n_components=2).fit_transform(pcs)
# dfnew['umap_x'] = Zspace[:,0]
# dfnew['umap_y'] = Zspace[:,1]

# # UMAPs more
# nns = [30, 60, 120, 240, 480]
# for nn in nns:
#     Zspace = umap.UMAP(n_components=2, n_neighbors=nn).fit_transform(pcs)
#     dfnew[f'umap_x_nn{nn}'] = Zspace[:,0]
#     dfnew[f'umap_y_nn{nn}'] = Zspace[:,1]

# # clsts
# numclsts = [2, 5, 10]
# for k in numclsts: 
#     model = KMeans(n_clusters=k)
#     clsts = model.fit_predict(pcs)
#     dfnew[f'clsts_k{k}'] = np.char.add("c", clsts.astype(str))

# # clsts
# G = miscu.build_feature_graph_knnlite(pcs, k=15, metric='euclidean')
# rs = [0.01, 0.02, 0.05, 0.1, 1]
# for r in rs:
#     clsts = miscu.leiden(G, dfnew.index.values, resolution=r) 
#     dfnew[f'clsts_r{r}'] = np.char.add("c", np.array(clsts).astype(str))

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(6*1,4*2))
ax = axs[0]
sns.boxplot(data=dfnew[ftrs_cols], ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_ylabel('counts')
ax.set_xlabel('Genes')

ax = axs[1]
sns.boxplot(data=dfnew[ftrs_cols], ax=ax)
ax.set_xticklabels([gene_names[ftr_col] for ftr_col in ftrs_cols], rotation=90)
ax.set_ylim([0,50])
ax.set_ylabel('counts')
ax.set_xlabel('Genes')

In [ ]:
# binary analysis
# ybins = np.linspace(0,400,100)
# dfnew['ybin'] = pd.cut(dfnew['y'], bins=ybins)

# xbins = np.linspace(0,400,100)
# dfnew['xbin'] = pd.cut(dfnew['x'], bins=xbins)

# zbins = np.linspace(0,400,100)
# dfnew['zbin'] = pd.cut(dfnew['z'], bins=zbins)

# bins_zb = np.linspace(0,400,9).astype(int)
# print(bins_zb)
# dfnew['bins_zb'] = pd.cut(dfnew['z'], bins=bins_zb)

# bins_zb_4p = np.linspace(0,400,5).astype(int)
# print(bins_zb_4p)
# dfnew['bins_zb_4p'] = pd.cut(dfnew['z'], bins=bins_zb_4p)


bins_yb = np.linspace(0,max_y,8+1).astype(int)
print(bins_yb)
dfnew['bins_yb'] = pd.cut(dfnew['y'], bins=bins_yb)

bins_yb_4p = np.linspace(0,max_y,4+1).astype(int)
print(bins_yb_4p)
dfnew['bins_yb_4p'] = pd.cut(dfnew['y'], bins=bins_yb_4p)

# z-sectioning visuals
- bin into zbin
- plot for each zbin

In [ ]:
def gen_palette(dfnew, col):
    # set global colorbar
    hue_order = np.unique(dfnew[col]) 
    ncat = len(hue_order)
    if ncat > 5:
        cmap = 'Paired'
    else:
        cmap = 'tab10'
    palette = {clst: clr for clst, clr in zip(hue_order, sns.color_palette(cmap, ncat))}
    return palette

In [ ]:
def view_z_sections(dfnew, sp_x, sp_y, groupby='bins_zb', col=None, cmap='coolwarm', title=None, vmaxp=95, vminp=0, 
                    figsize=(4*4,2*4),
                   ):
    """
    """
    # assert len(dfnew['bins_zb'].unique())-1 == 8 # 8 intervals
    
    # set global colorbar
    if col is not None:
        vmax = np.percentile(dfnew[col].values, vmaxp)
        vmin = -vmax*0.1
        # vmin = np.percentile(dfnew[col].values, vminp)
        
    fig, axs = plt.subplots(2, 4, figsize=figsize, sharex=True, sharey=True)
    for i, (thisbin, dfsub) in enumerate(dfnew.groupby(groupby)):
        ax = axs.flat[i]
        ax.grid(False)

        x = dfsub[sp_x].values
        y = dfsub[sp_y].values
        if col is not None:
            c = dfsub[col].values
            g = ax.scatter(x, y, c=c, s=10, cmap=cmap, vmax=vmax, vmin=vmin,
                           edgecolor='none', 
                          )
        else:
            g = ax.scatter(x, y, s=10, c='gray', 
                           edgecolor='none', 
                          )
            
        sns.despine(ax=ax)
        ax.set_aspect('equal')
        ax.set_title(f" z: {thisbin}", loc='left', y=0.95)#va='top')#va='top', y=0.96)

        if i%4 == 0:
            ax.set_ylabel(sp_y)
        if i >= 4:
            ax.set_xlabel(sp_x)

    if col is not None:
        cax = fig.add_axes([0.93, 0.3, 0.01, 0.3])
        fig.colorbar(g, cax=cax, label='', shrink=0.3, aspect=10, ticks=[0,vmax], orientation='vertical')
    fig.subplots_adjust(hspace=0.1, wspace=0.05)
    if title is not None:
        fig.suptitle(title, fontsize=20, y=0.92)
    plt.show()

In [ ]:
def view_z_sections_4panels(dfnew, sp_x, sp_y, groupby='bins_zb_4p', 
                            col=None, cmap='coolwarm', title=None, vmaxp=95, vminp=0,
                            figsize=(4*4,1*4),
                           ):
    """
    """
    # assert len(dfnew['bins_zb_4p'].unique())-1 == 4 # 4 intervals
    
    # set global colorbar
    if col is not None:
        vmax = np.percentile(dfnew[col].values, vmaxp)
        vmin = 0 # -vmax*0.1
        # vmin = np.percentile(dfnew[col].values, vminp)
        
    fig, axs = plt.subplots(1, 4, figsize=figsize, sharex=True, sharey=True)
    for i, (thisbin, dfsub) in enumerate(dfnew.groupby(groupby)):
        ax = axs.flat[i]
        ax.grid(False)

        x = dfsub[sp_x].values
        y = dfsub[sp_y].values
        if col is not None:
            c = dfsub[col].values
            cond = c > 1e-5
            g = ax.scatter(x[~cond], y[~cond],  
                           s=8, 
                           color='lightgray', #cmap=cmap, vmax=vmax, vmin=vmin,
                           edgecolor='none', 
                           zorder=0,
                          )
            g = ax.scatter(x[cond], y[cond], c=c[cond], s=8, cmap=cmap, vmax=vmax, vmin=vmin,
                           edgecolor='none', 
                           zorder=2,
                          )
        else:
            g = ax.scatter(x, y, s=8, c='gray', 
                           edgecolor='none', 
                          )
            
        sns.despine(ax=ax)
        ax.set_aspect('equal')
        ax.set_title(f" z: {thisbin}", loc='left', y=0.95)#va='top')#va='top', y=0.96)

        if i == 0:
            ax.set_ylabel(sp_y)
        ax.set_xlabel(sp_x)

    if col is not None:
        cax = fig.add_axes([0.93, 0.3, 0.01, 0.3])
        fig.colorbar(g, cax=cax, label='', shrink=0.3, aspect=10, ticks=[0,vmax], orientation='vertical')
    fig.subplots_adjust(hspace=0.1, wspace=0.05)
    if title is not None:
        fig.suptitle(title, fontsize=20, y=0.92)
    plt.show()

In [ ]:
def view_z_sections_4panels_binary(dfnew, sp_x, sp_y, groupby='bins_zb_4p', 
                            col=None, cmap='coolwarm', title=None, vmaxp=95, vminp=0,
                            figsize=(4*4,1*4),
                           ):
    """
    """
    # assert len(dfnew['bins_zb_4p'].unique())-1 == 4 # 4 intervals
    
    # set global colorbar
    if col is not None:
        vmax = np.percentile(dfnew[col].values, vmaxp)
        vmin = 0 # -vmax*0.1
        # vmin = np.percentile(dfnew[col].values, vminp)
        
    fig, axs = plt.subplots(1, 4, figsize=figsize, sharex=True, sharey=True)
    for i, (thisbin, dfsub) in enumerate(dfnew.groupby(groupby)):
        ax = axs.flat[i]
        ax.grid(False)

        x = dfsub[sp_x].values
        y = dfsub[sp_y].values
        if col is not None:
            c = dfsub[col].values
            cond = c > 0
            g = ax.scatter(x[~cond], y[~cond], s=8, #cmap=cmap, vmax=vmax, vmin=vmin,
                           color='lightgray',
                           edgecolor='none', 
                           zorder=0,
                          )
            g = ax.scatter(x[cond], y[cond], s=20, # cmap=cmap, vmax=vmax, vmin=vmin,
                           color='C1',
                           edgecolor='none', 
                           zorder=2,
                           label=f'{col} > 0',
                          )
        else:
            g = ax.scatter(x, y, s=8, c='gray', 
                           edgecolor='none', 
                          )
            
        sns.despine(ax=ax)
        ax.set_aspect('equal')
        ax.set_title(f" z: {thisbin}", loc='left', y=0.95)#va='top')#va='top', y=0.96)

        if i == 0:
            ax.set_ylabel(sp_y)
        ax.set_xlabel(sp_x)

    ax.legend()
    fig.subplots_adjust(hspace=0.1, wspace=0.05)
    if title is not None:
        fig.suptitle(title, fontsize=20, y=0.92)
    plt.show()

In [ ]:
def view_z_sections_labels(dfnew, sp_x, sp_y, col, groupby='bins_zb', palette=None, title=None):
    """
    """
    # assert len(dfnew['bins_zb'].unique())-1 == 8 # 8 intervals
    
    # set global colorbar
    hue_order = np.unique(dfnew[col]) 
    ncat = len(hue_order)
    if ncat > 5:
        cmap = 'Paired'
    else:
        cmap = 'tab10'
    if palette is None:
        palette = {clst: clr for clst, clr in zip(hue_order, sns.color_palette(cmap, ncat))}
        
    fig, axs = plt.subplots(2, 4, figsize=(4*4,2*4), sharex=True, sharey=True)
    for i, (thisbin, dfsub) in enumerate(dfnew.groupby(groupby)):
        ax = axs.flat[i]
        ax.grid(False)
        if i == 4-1:
            legend = True
        else:
            legend = False
        g = sns.scatterplot(data=dfsub, x=sp_x, y=sp_y, hue=col, 
                            hue_order=hue_order, palette=palette,
                            s=10, 
                            edgecolor='none', 
                            ax=ax,
                            legend=legend,
                          )
        sns.despine(ax=ax)
        ax.set_aspect('equal')
        ax.set_title(f" z: {thisbin}", loc='left', y=0.95)#va='top')#va='top', y=0.96)

        if i%4 == 0:
            ax.set_ylabel(sp_y)
        if i >= 4:
            ax.set_xlabel(sp_x)
        if i == 4-1:
            ax.legend(bbox_to_anchor=(1,1))

    fig.subplots_adjust(hspace=0.1, wspace=0.05)
    if title is not None:
        fig.suptitle(title, fontsize=20, y=0.92)
    plt.show()

In [ ]:
col = ftrs_order[0]
title = gene_names[col]

sp_x, sp_y, groupby = 'x', 'z', 'bins_yb'
view_z_sections(dfnew, sp_x, sp_y, groupby=groupby, col=col, cmap='Greys', title=title)

sp_x, sp_y, groupby = 'x', 'z', 'bins_yb_4p'
view_z_sections_4panels(dfnew, sp_x, sp_y, groupby=groupby, col=col, cmap='Greys', title=title)

In [ ]:
for col in ftrs_order:
    # col = ftrs_order[0]
    title = gene_names[col]

    # sp_x, sp_y, groupby = 'x', 'z', 'bins_yb'
    # view_z_sections(dfnew, sp_x, sp_y, groupby=groupby, col=col, cmap='Greys', title=title)

    sp_x, sp_y, groupby = 'x', 'z', 'bins_yb_4p'
    view_z_sections_4panels(dfnew, sp_x, sp_y, groupby=groupby, col=col, cmap='Greys', title=title, figsize=(4*6,1*4))

In [ ]:
plt.plot(np.sort(dfnew['gcamp_binary']))

In [ ]:
a = dfnew['gcamp']
a = a[a>100]
plt.plot(np.sort(a)[::-1])

In [ ]:
sp_x, sp_y, groupby = 'x', 'z', 'bins_yb_4p'
col = 'gcamp_binary'
title = 'GCaMP cell'
view_z_sections_4panels_binary(dfnew, sp_x, sp_y, groupby=groupby, col=col, cmap='Greys', title=title, figsize=(4*6,1*4), vmaxp=100)

In [ ]:
dfnew[dfnew['gcamp_binary']>0].sort_values('z', ascending=False)[['r1_c2', 'r1_c0', 'r1_c4', 'z', 'x', 'y', 'gcamp']] #[[]]

In [ ]:
df[df['gcamp_binary']>0].sort_values('z', ascending=False)[['r1_c2', 'r1_c0', 'r1_c4', 'z', 'x', 'y', 'gcamp']] #[[]]

# hierarchical clustering

In [ ]:
def gen_discrete_colors(N):
    """
    """
    fig, ax = plt.subplots(figsize=(6, 1))
    fig.subplots_adjust(bottom=0.5)
    cmap = matplotlib.cm.jet
    norm = matplotlib.colors.BoundaryNorm(1+np.arange(N), cmap.N, extend='max')
    dismap = matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap)
    fig.colorbar(dismap,
                 cax=ax, orientation='horizontal')
    plt.show()
    
    palette = {i: dismap.to_rgba(i)[:3] for i in 1+np.arange(N)}
    plt.figure(figsize=(5,1))
    plt.scatter(1+np.arange(N), [0]*N, c=[palette[p] for p in palette.keys()])
    plt.show()
    return palette, dismap

In [ ]:
# sch
col_link = sch.linkage(mat_ftrs.T, method='ward', metric='euclidean')
col_order = sch.dendrogram(col_link, no_plot=True)['leaves']

row_link = sch.linkage(mat_ftrs, method='ward', metric='euclidean')
row_order = sch.dendrogram(row_link, p=30, truncate_mode='none', no_plot=True)['leaves']

# get clusters
# clsts1 = sch.fcluster(row_link, 100, criterion='distance')
# clsts2 = sch.fcluster(row_link,  50, criterion='distance')
# clsts3 = sch.fcluster(row_link,  30, criterion='distance')
clsts1 = sch.fcluster(row_link, 200, criterion='distance')
clsts2 = sch.fcluster(row_link, 150, criterion='distance')
clsts3 = sch.fcluster(row_link, 100, criterion='distance')

print(clsts1.shape, np.unique(clsts1).shape)
print(clsts2.shape, np.unique(clsts2).shape)
print(clsts3.shape, np.unique(clsts3).shape)

dfnew['clst_l1'] = clsts1
dfnew['clst_l2'] = clsts2
dfnew['clst_l3'] = clsts3

# colors
palette1, dismap1 = gen_discrete_colors(len(np.unique(clsts1)))
palette2, dismap2 = gen_discrete_colors(len(np.unique(clsts2)))
palette3, dismap3 = gen_discrete_colors(len(np.unique(clsts3)))

In [ ]:
sp_x, sp_y = 'x', 'z'
fig, axs = plt.subplots(1,3,figsize=(4*3,5*1), sharey=True, sharex=True)
# cbar_ax = fig.add_axes([0.92, 0.5, 0.01, 0.2])
for i, (col, palette, dismap, ax) in enumerate(zip(['clst_l1', 'clst_l2', 'clst_l3'], 
                                                   [ palette1,  palette2,  palette3],
                                                   [ dismap1,   dismap2,   dismap3],
                                                   axs.flat)):
    x = dfnew[sp_x].values
    y = dfnew[sp_y].values
    c = dfnew[col].values
    
    ax.grid(False)
    g = ax.scatter(x, y, c=pd.Series(palette)[c], s=5, edgecolor='none',)
    sns.despine(ax=ax)
    ax.set_title(col)
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    ax.axis('off')
    fig.colorbar(dismap, ax=ax, orientation='horizontal', shrink=0.5, aspect=10)
    
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
genes_order_v2 = np.hstack([
    mat_ftrs.columns[col_order],
    # mat_cre.columns,
    # ['RL'],
])
ftrs_order_v2 = channel_names.loc[genes_order_v2].values
ftrs_order_v2

In [ ]:
fig = plt.figure(figsize=(10,15))
axd = fig.subplot_mosaic("A"*3+"BEF")
ax = axd["A"]
sns.heatmap(
    mat_ftrs.iloc[row_order,col_order],
    yticklabels=False,
    cmap='coolwarm',
    cbar_kws=dict(shrink=0.5, location='bottom', label='zscore log2(1+normed counts)'),
    vmax=3, vmin=-3,
    ax=ax,
)
ax.set_ylabel('Cells')

for i, (ax, col) in enumerate(zip(
    (axd["B"],),# axd["C"], axd["D"]),
    ('clst_l1',),# 'clst_l2', 'clst_l3'),
    )):
    vec = dfnew[col].values
    sns.heatmap(
        dfnew[[col]].rename(columns={col: col.replace('clst_l', 'L')}).iloc[row_order],
        yticklabels=False,
        cmap='jet',
        cbar_kws=dict(location='bottom', aspect=5, label=''),
        ax=ax,
    )
    ax.set_ylabel('')
    positions = np.cumsum(np.unique(vec, return_counts=True)[1])
    ax.hlines(positions, xmin=-0.2, xmax=1, linewidth=1, linestyle='-', color='k', clip_on=False)
    for i, pos in enumerate(positions):
        ax.text(0, pos, i+1, color='white')

# ax = axd["E"]
# sns.heatmap(
#     mat_cre.iloc[row_order],
#     yticklabels=False,
#     cmap='Greys',
#     cbar_kws=dict(location='bottom', aspect=5, label='log2(1+normed counts)'),
#     ax=ax,
# )
# ax.set_ylabel('')

ax = axd["F"]
sns.heatmap(
    dfnew[['z']].iloc[row_order], 
    yticklabels=False,
    cmap='Greys',
    cbar_kws=dict(location='bottom', aspect=5, label='um'),
    # vmin=0,
    ax=ax,
)
ax.set_ylabel('')
fig.tight_layout()

In [ ]:
sp_x, sp_y = 'x', 'z'
col = 'clst_l1'
palette = palette1
dismap = dismap1
n = len(dfnew[col].unique())
nx = min(3,n)
ny = int((n+nx-1)/nx)
suptitle = 'L1'

fig, axs = plt.subplots(ny,nx,figsize=(3*nx,3*ny), sharey=True, sharex=True)
for i, ax in enumerate(axs.flat):
    x = dfnew[sp_x].values
    y = dfnew[sp_y].values
    c = dfnew[col].values
    
    ax.grid(False)
    g = ax.scatter(x, y, c=[palette[_c] if i+1==_c else 'lightgray' for _c in c], s=5, edgecolor='none',)
    
    sns.despine(ax=ax)
    ax.set_title(f"C{i+1}")
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    ax.axis('off')
    
fig.suptitle(suptitle)
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
sp_x, sp_y = 'x', 'z'
col = 'clst_l2'
palette = palette2
dismap = dismap2
n = len(dfnew[col].unique())
nx = min(4,n)
ny = int((n+nx-1)/nx)
suptitle = 'L2'

fig, axs = plt.subplots(ny,nx,figsize=(3*nx,3*ny), sharey=True, sharex=True)
for i, ax in enumerate(axs.flat):
    x = dfnew[sp_x].values
    y = dfnew[sp_y].values
    c = dfnew[col].values
    
    ax.grid(False)
    g = ax.scatter(x, y, c=[palette[_c] if i+1==_c else 'lightgray' for _c in c], s=5, edgecolor='none',)
    
    sns.despine(ax=ax)
    ax.set_title(f"C{i+1}")
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    ax.axis('off')
    
fig.suptitle(suptitle)
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
ftrs_order_v2

In [ ]:
sp_x, sp_y = 'x', 'z'
fig, axs = plt.subplots(4,4,figsize=(6*4,4*4), sharey=True, sharex=True)
cbar_ax = fig.add_axes([0.92, 0.5, 0.01, 0.2])
for i, (col, ax) in enumerate(zip(ftrs_order[::-1], axs.flat)):
    x = dfnew[sp_x].values
    y = dfnew[sp_y].values
    c = dfnew[col].values
    vmax=np.percentile(c, 95)+1e-7
    
    ax.axis('off')
    ax.grid(False)
    g = ax.scatter(x, y, c=c/vmax, s=5, edgecolor='none', cmap='Greys', vmax=1, vmin=-0.1) #vmax, vmin=-0.1*vmax)
    sns.despine(ax=ax)
    ax.set_title(gene_names[col.replace('_log2', '')])
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')

for ax in axs.flat[i:]:
    ax.axis('off')
    
fig.colorbar(g, cax=cbar_ax, label='Normed counts\n(0-95 perctl.)', aspect=5, shrink=0.3, ticks=[0, 1])
fig.subplots_adjust(hspace=0.1, wspace=0.02)

# co-expression, clustering, cluster enrichment of expression

In [ ]:
sp_x, sp_y, groupby = 'x', 'z', 'bins_yb'
col = 'clst_l1'
view_z_sections_labels(dfnew, sp_x, sp_y, col, groupby=groupby, palette=palette1, title=None)

In [ ]:
group = 'clst_l1'
palette = palette2
data = dfnew

fig, axs = plt.subplots(6,1,figsize=(8*1,4*6), sharex=True)
grouporder = np.unique(data[group])
for i, col in enumerate(ftrs_order_v2):
    ax = axs.flat[i]
    # col = ftrs_order[0]

    sns.boxplot(data=data, x=group, order=grouporder, palette=palette, y=col, ax=ax,)
    sns.despine(ax=ax)
    ax.set_ylim(ymin=0.5)
    ax.set_title(gene_names[col])
    ax.set_ylabel("") #gene_names[col])
    ax.set_xlabel("") #gene_names[col])
    ax.set_yscale('log')
fig.subplots_adjust(wspace=0.3)

In [ ]:
group = 'clst_l1'
dfmean = dfnew.groupby(group)[ftrs_order_v2].mean()
dfmean = basicu.zscore(dfmean, axis=0)

fig, ax = plt.subplots(figsize=(6,4))
sns.heatmap(dfmean.T, ax=ax, 
            yticklabels=[gene_names[col] for col in dfmean.columns],
            cmap='coolwarm', cbar_kws=dict(label='zscore mean exp.', shrink=0.5, aspect=10))